In [ ]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager
import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
import feather
import uuid
import sys
import time
sys.path.append('../..')
from slp.slp_package.slp_functions import one_hot_encode_flags

In [ ]:
df = pd.read_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\all_game_data_df.parquet")

In [ ]:
print(df.columns)
# print(df.info())
df.head()


In [ ]:
filtered_df = df.loc[df['is_teams'] == False]
filtered_df = filtered_df.loc[filtered_df['num_players'] == 2]
filtered_df = filtered_df.loc[filtered_df['player_1_type_name'] == 'HUMAN']
filtered_df = filtered_df.loc[filtered_df['player_2_type_name'] == 'HUMAN']
filtered_df = filtered_df.loc[filtered_df['conclusive'] == True]
filtered_df = filtered_df.loc[filtered_df['source'] == 'mango']
reduced_df = filtered_df[['slippi','source','length','player_1_character_name','player_2_character_name','player_1_display_name','player_2_display_name','player_1_netplay_code','player_2_netplay_code','stage_name','winning_player','player_1_did_win','player_2_did_win','player_1_inputs_np_save_path','player_2_inputs_np_save_path']]
# Assuming filtered_df is your DataFrame
# Concatenate the two columns into one Series
all_netplay_codes = pd.concat([filtered_df['player_1_netplay_code'], filtered_df['player_2_netplay_code']])

# Count the occurrences of each netplay code
code_counts = all_netplay_codes.value_counts()

# Display the top 10 most common values
top_10 = code_counts.head(20)
print(top_10)
reduced_df.head()

In [ ]:
# print(df['is_teams'].unique())
reduced_df.info()

In [ ]:
input_np_path = os.path.join(df['inputs_np_common_path'][10],df['player_2_inputs_np_sub_path'][10])
with gzip.open(input_np_path, 'rb') as f:
    input_np = np.load(f)


print(input_np)

In [ ]:
# def process_file(path,time_1,time_2,time_3):
#     save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    
#     time_1_start = time.time()
#     frame_data_df = pd.read_feather(path)
#     time_1.append(time.time()-time_1_start)
    
#     time_2_start = time.time()
#     inputs = np.empty((4,frame_data_df.shape[0]),dtype = float)
#     for i in range(frame_data_df.shape[0]):
#         inputs[0, i] = frame_data_df['pre_joystick_x'][i]
#         inputs[1, i] = frame_data_df['pre_joystick_y'][i]
#         inputs[2, i] = frame_data_df['pre_cstick_x'][i]
#         inputs[3, i] = frame_data_df['pre_cstick_y'][i] 
#         # inputs[4:,i] = one_hot_encode_flags(slp.event.Buttons.Physical,frame_data_df['pre_buttons_physical'][i])
#     time_2.append(time.time()-time_2_start)
    
#     time_3_start = time.time()
#     full_path = os.path.join(save_path, str(uuid.uuid4()) + '.npy.gz')
#     with gzip.open(full_path, 'wb') as f:
#         np.save(f, inputs)
#     time_3.append(time.time() - time_3_start)
#     return inputs         


def process_file(path,time_1,time_2,time_3,buttons):
    save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    time_1_start = time.time()
    frame_data_df = pd.read_parquet(path,engine='pyarrow')
    time_1.append(time.time()-time_1_start)
    num_rows = frame_data_df.shape[0]

    # Assuming one_hot_encode_flags can process a series and return a 2D array
    # encoded_flags = one_hot_encode_flags(slp.event.Buttons.Physical, frame_data_df['pre_buttons_physical'])


    time_2_start = time.time()

    # button_states = frame_data_df['pre_buttons_logical'].to_numpy()[:, None]
    # bitmasks = np.array([button.value for button in buttons if button != buttons.NONE])[None, :]
    # encoded = (button_states & bitmasks) != 0
    # inputs = np.empty((num_rows,25), dtype=float)
    # inputs[ :,0] = frame_data_df['pre_joystick_x']
    # inputs[ :,1] = frame_data_df['pre_joystick_y']
    # inputs[ :,2] = frame_data_df['pre_cstick_x']
    # inputs[ :,2] = frame_data_df['pre_cstick_y']
    # inputs[:,4:] = encoded
    # inputs.T
    
    # Convert the 'pre_buttons_logical' column to a NumPy array
    button_states = frame_data_df['pre_buttons_logical'].values

    # For each button, apply the bitmask and add a column to the DataFrame
    for button in buttons:
        if button == buttons.NONE:  # Skip the NONE value to avoid an unnecessary column
            continue
        # Use broadcasting to apply the bitmask and generate the one-hot encoded array
    frame_data_df[button.name] = (button_states & button.value) != 0
    
    time_2.append(time.time()-time_2_start)
    
    time_3_start = time.time()
    # full_path = os.path.join(save_path, f"{uuid.uuid4()}.npy.gz")
    # with gzip.open(full_path, 'wb') as f:
    #     np.save(f, inputs)
    time_3.append(time.time() - time_3_start)
    
    return 

In [ ]:
# process_file(path_list[0],1)
manager = Manager()
time_1 = manager.list()
time_2 = manager.list()
time_3 = manager.list()

buttons = slp.event.Buttons.Logical
        
# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=1)(delayed(process_file)(path,time_1,time_2,time_3,buttons) for path in tqdm.tqdm(path_list))



In [ ]:
print(sum(time_1))
print(sum(time_2))
print(sum(time_3))


In [ ]:
frame_data_df =   pd.read_parquet(path_list[1],engine='pyarrow')

button_states = frame_data_df['pre_buttons_logical'].values

# For each button, apply the bitmask and add a column to the DataFrame
for button in buttons:
    if button == buttons.NONE:  # Skip the NONE value to avoid an unnecessary column
        continue
    # Use broadcasting to apply the bitmask and generate the one-hot enco
    # ed array
    frame_data_df[button.name] = (button_states & button.value) != 0

In [ ]:
def np_pd_load_test(df_row,time_1,time_2):
    path_df  = os.path.join(df_row['inputs_df_common_path'],df_row['player_2_inputs_df_sub_path'])
    path_np = os.path.join(df_row['inputs_np_common_path'],df_row['player_2_inputs_np_sub_path'])
    
    time_1_start = time.time()
    frame_data_df = pd.read_parquet(path_df,engine='pyarrow')
    time_1.append(time.time()-time_1_start)


    time_2_start = time.time()
    with gzip.open(path_np, 'rb') as f:
        input_np = np.load(f)
    time_2.append(time.time()-time_2_start)
    
    return 

# process_file(path_list[0],1)
manager = Manager()
time_1 = manager.list()
time_2 = manager.list()
        
# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=1)(delayed(np_pd_load_test)(df_row,time_1,time_2) for index, df_row in tqdm.tqdm(df.iterrows()))

print(sum(time_1))
print(sum(time_2))